# Agenda

1. The iterator protocol
    - Adding iteration to your classes
    - Different techniques for that
2. Generator functions
3. Generator comprehensions
4. Decorators
5. Threading and multiprocessing 

In [1]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def greet(self):
        return f'Hello, {self.name}!'
        
p1 = Person('name1')        
p2 = Person('name2')

print(p1.greet())
print(p2.greet())

Hello, name1!
Hello, name2!


In [2]:
import weakref

In [3]:
p3 = weakref.ref(p2)


In [4]:
p3

<weakref at 0x10c289310; to 'Person' at 0x10c2874c0>

In [5]:
p3.ref

AttributeError: 'weakref' object has no attribute 'ref'

In [7]:
w = weakref.ref(Person('name10'))

In [8]:
w

<weakref at 0x10b2aabd0; dead>

In [9]:
type(w)

weakref

In [10]:
dir(w)

['__call__',
 '__callback__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [11]:
p3 = weakref.ref(p2)


In [13]:
p3

<weakref at 0x10c289310; to 'Person' at 0x10c2874c0>

In [14]:
p1 = Person('name1')
p2 = Person('name2')

all_people = {1: p1, 2: p2}

In [15]:
del(p1)

In [16]:
len(all_people)

2

In [17]:
p1 = Person('name1')
p2 = Person('name2')

all_people = weakref.WeakValueDictionary()

all_people[1] = p1
all_people[2] = p2

In [18]:
all_people

In [20]:
list(all_people.items())

[(1, <__main__.Person at 0x10c263760>), (2, <__main__.Person at 0x10c2636a0>)]

In [21]:
len(all_people)

2

In [22]:
del(p1)

In [23]:
len(all_people)

2

In [24]:
len(all_people)

2

In [25]:
list(all_people.items())

[(1, <__main__.Person at 0x10c263760>), (2, <__main__.Person at 0x10c2636a0>)]

In [30]:
p1 = Person('name1')
p2 = Person('name2')

all_people = weakref.WeakValueDictionary()

all_people[1] = p1
all_people[2] = p2

In [31]:
del(p1)

In [32]:
len(all_people)

1

In [33]:
del(p2)

In [34]:
len(all_people)

0

# Iterator protocol

In [35]:
s = 'abcde'

for one_character in s:
    print(one_character)

a
b
c
d
e


# Protocol

1. Ask an object if it's iterable (`iter`)
    - Returns an iterator object if it is iterable
    - Raises an exception if it's not iterable
2. Ask the iterator we got back for its next item (`next`)
3. Each time we call `next`, we'll get an object
4. When we reach the end of the iteration, we get... the `StopIteration` exception

In [36]:
iter(s)

In [37]:
iter(s)

In [38]:
iter(s)

In [39]:
i = iter(s)  # store the string iterator in "i"

In [40]:
next(i)

'a'

In [41]:
next(i)

'b'

In [42]:
next(i)

'c'

In [43]:
next(i)

'd'

In [44]:
next(i)

'e'

In [45]:
next(i)

StopIteration: 

In [46]:
for i in 10:
    print(i)

TypeError: 'int' object is not iterable

In [47]:
iter(10)

TypeError: 'int' object is not iterable

In [55]:
class MyIterator:
    def __init__(self, data):
        print(f'Now in MyIterator.__init__ with {data=}')
        self.data = data
        self.index = 0
        
    def __iter__(self):
        print(f'Now in MyIterator.__iter__')
        return self   # the object is its own iterator!
    
    def __next__(self):
        print(f'Now entering MyIterator.__next__')
        if self.index >= len(self.data):
            print(f'\t{self.index=}, ending the loop')
            raise StopIteration
            
        value = self.data[self.index]
        print(f'\t{self.index=}, {value=}')
        self.index += 1
        return value

m = MyIterator('abc')

for one_item in m:
    print(one_item)

Now in MyIterator.__init__ with data='abc'
Now in MyIterator.__iter__
Now entering MyIterator.__next__
	self.index=0, value='a'
a
Now entering MyIterator.__next__
	self.index=1, value='b'
b
Now entering MyIterator.__next__
	self.index=2, value='c'
c
Now entering MyIterator.__next__
	self.index=3, ending the loop


In [51]:
f = open('/etc/passwd')

iter(f) is f

True

# Exercise: Circle

1. Create a class, `Circle`, which takes two arguments:
    - an iterable piece of data (`data`)
    - an integer (`maxtimes`)
2. If we run a `for` loop on an instance of `Circle`, we'll get `maxtimes` results back.
3. The return values will come from `data`.
    - If the number of elements in `data` is larger than `maxtimes`, then we'll just end after `maxtimes`, producing one value at a time.
    - If the number of elements in `data` is smaller than `maxtimes`, then when we get to the end of the data, we'll come back to index 0 -- thus going around and around until we give the number of results indicated in `maxtimes`.
    
```python
c = Circle('abc', 7)

for one_item in c:
    print(one_item)
```

```
a
b
c
a
b
c
a
```

In [57]:
class Circle:
    def __init__(self, data, maxtimes):
        self.data = data
        self.maxtimes = maxtimes
        self.index = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.index >= self.maxtimes:
            raise StopIteration
            
        value = self.data[self.index % len(self.data)]
        self.index += 1
        return value
    
c = Circle('abc', 7)

for one_item in c:
    print(one_item)

a
b
c
a
b
c
a


In [58]:
0 % 3  # remainder from division of 0/3

0

In [59]:
1 % 3  # remainder from 1/3

1

In [60]:
2 % 3

2

In [61]:
3 % 3

0

In [62]:
4 % 3

1

In [63]:
5 % 3

2

In [64]:
mylist = [10, 20, 30, 40, 50]

i1 = iter(mylist)
i2 = iter(mylist)

In [65]:
i1

In [66]:
i2

In [67]:
next(i1)

10

In [68]:
next(i2)

10

In [69]:
next(i2)

20

In [71]:
next(i2)

30

In [72]:
next(i1)

20

In [74]:
c = Circle('abc', 7)

i1 = iter(c)
i2 = iter(c)

In [75]:
next(i1)

'a'

In [76]:
next(i1)

'b'

In [77]:
next(i2)

'c'

In [78]:
next(i2)

'a'

In [79]:
next(i1)

'b'

In [80]:
i1 is i2

True

In [81]:
i1 is c

True

In [83]:
c = Circle('abc', 7)

print('*** A ****')
for one_item in c:
    print(one_item, end= ' ')
    
print()

print('*** B ****')
for one_item in c:
    print(one_item, end= ' ')


*** A ****
a b c a b c a 
*** B ****


In [88]:
class CircleIterator:
    def __init__(self, data, maxtimes):
        self.data = data
        self.maxtimes = maxtimes
        self.index = 0

    def __next__(self):
        if self.index >= self.maxtimes:
            raise StopIteration
            
        value = self.data[self.index % len(self.data)]
        self.index += 1
        return value
    
class Circle:
    def __init__(self, data, maxtimes):
        self.data = data
        self.maxtimes = maxtimes
        
    def __iter__(self):
        return CircleIterator(self.data, self.maxtimes)
    
    
c = Circle('abc', 7)

print("*** A ****")
for one_item in c:
    print(one_item)
    
print("*** B ****")
for one_item in c:
    print(one_item)
    


*** A ****
a
b
c
a
b
c
a
*** B ****
a
b
c
a
b
c
a


In [85]:
iter(c)

In [86]:
iter(c)

In [87]:
iter(c)

In [89]:
class CircleIterator:
    def __init__(self, circle):
        self.circle = circle
        self.index = 0

    def __next__(self):
        if self.index >= self.circle.maxtimes:
            raise StopIteration
            
        value = self.circle.data[self.index % len(self.circle.data)]
        self.index += 1
        return value
    
class Circle:
    def __init__(self, data, maxtimes):
        self.data = data
        self.maxtimes = maxtimes
        
    def __iter__(self):
        return CircleIterator(self)
    
    
c = Circle('abc', 7)

print("*** A ****")
for one_item in c:
    print(one_item)
    
print("*** B ****")
for one_item in c:
    print(one_item)
    


*** A ****
a
b
c
a
b
c
a
*** B ****
a
b
c
a
b
c
a


In [91]:
# what if we set self.index = 0 just before raising the exception?

class Circle:
    def __init__(self, data, maxtimes):
        self.data = data
        self.maxtimes = maxtimes
        self.index = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.index >= self.maxtimes:
            self.index = 0
            raise StopIteration
            
        value = self.data[self.index % len(self.data)]
        self.index += 1
        return value
    
c = Circle('abc', 7)

print(f'*** A ***')
for one_item in c:
    print(one_item)

print(f'*** B ***')  
for one_item in c:
    print(one_item)

*** A ***
a
b
c
a
b
c
a
*** B ***
a
b
c
a
b
c
a


In [92]:
c = Circle('abc', 7)

i1 = iter(c)
i2 = iter(c)

In [93]:
i1 is i2

True

In [94]:
next(i1)

'a'

In [95]:
next(i1)

'b'

In [96]:
next(i2)

'c'

In [97]:
next(i2)

'a'

# Exercise: MyRange

1. Create a class, `MyRange`, which takes 1, 2, or 3 arguments.  It'll work much like `range` does.
2. If we call it with 1 argument, then we expect the iterations to run from 0 up to (and not including) that number.
3. If we call it with 2 arguments, then we expect the iterations to run from the first number up to (and not including) the second.
4. If we call it with 3 arguments, then we expect the iterations to run from the first to the second, with a step size of the third.
5. Use the two-class method for iterator construction in creating this.

In [98]:
list(range(5))

[0, 1, 2, 3, 4]

In [99]:
list(range(5, 10))

[5, 6, 7, 8, 9]

In [100]:
list(range(5, 20, 2))

[5, 7, 9, 11, 13, 15, 17, 19]

In [106]:
class MyRangeIterator:
    def __init__(self, current, stop, step):
        self.current = current
        self.stop = stop
        self.step = step
    
    def __next__(self):
        if self.current >= self.stop:
            raise StopIteration
            
        value = self.current
        self.current += self.step
        
        return value
    

class MyRange:
    def __init__(self, first, second=None, step=1):
        if second is None:
            self.current = 0
            self.stop = first
        else:
            self.current = first
            self.stop = second
            
        self.step = step
        
    def __iter__(self):
        return MyRangeIterator(self.current, self.stop, self.step)
    
list(MyRange(5))

[0, 1, 2, 3, 4]

In [107]:
list(MyRange(5, 10))

[5, 6, 7, 8, 9]

In [108]:
list(MyRange(5, 20, 2))

[5, 7, 9, 11, 13, 15, 17, 19]

In [102]:
list('abc')

['a', 'b', 'c']

In [103]:
list((10, 20, 309))

[10, 20, 309]

 # Generator functions

In [109]:
def myfunc():
    return 1
    return 2
    return 3

In [110]:
myfunc()

1

In [111]:
import dis
dis.dis(myfunc)

  2           0 LOAD_CONST               1 (1)
              2 RETURN_VALUE


In [112]:
def myfunc():
    yield 1
    yield 2
    yield 3

In [113]:
myfunc()

<generator object myfunc at 0x10ce27f90>

In [114]:
g = myfunc()

In [115]:
type(g)

generator

In [116]:
next(g)

1

In [117]:
next(g)

2

In [118]:
next(g)

3

In [119]:
next(g)

StopIteration: 

In [120]:
def myfunc():
    yield 1
    yield 2
    yield 3

Running `next` on a generator object executes the function body through the next `yield` statement. 

The returned value from `next` is whatever `yield` returned.

If we hit the end of the function body, the generator raises `StopIteration`.